# Yellow Taxi Trip Analysis (Jan 2025)

This notebook analyzes NYC Yellow Taxi Trip data for January 2025, sourced from the NYC TLC. The analysis covers trip details such as pickup/dropoff times, distances, fares, passenger behavior, payment methods, and location zones. 

Refer to the data dictionary: `data_dictionary_trip_records_yellow.pdf` or [NYC TLC](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml) for column definitions.

In [1]:
# Import libraries for data analysis and visualization
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Apply seaborn style for improved visualization aesthetics
plt.style.use('seaborn')

## 1. Data Loading and Initial Preview

Load the dataset and display the first few rows to understand its structure.

In [2]:
df = pd.read_parquet(r"data\yellow_tripdata_2025-01.parquet", engine="pyarrow")df.head()

## 2. Data Inspection

Check column types and counts.

In [3]:
df.info()

## 3. Data Cleaning

Handle missing, duplicates, invalid, and outlier values.

### 3.1 Missing Values

Assess missing data percentages.

In [4]:
missing = df.isnull().sum()missing_pct = (missing / len(df) * 100).round(2)pd.DataFrame({"Count": missing, "Percent": missing_pct})

**Note:** `passenger_count` and `RatecodeID` have ~15% missing—consider imputation later.

### 3.2 Duplicates

In [5]:
print(f"Duplicates found: {df.duplicated().sum()}")

### 3.3 Negative Values

Remove rows with any negative numeric values.

In [6]:
numeric_cols = df.select_dtypes(include=[np.number]).columnsfor col in numeric_cols:    neg = (df[col] < 0).sum()    if neg:        print(f"{col}: {neg} negative values")df = df[(df[numeric_cols] >= 0).all(axis=1)]

### 3.4 Zero Values

Filter out rows where key fields are zero.

In [7]:
non_zero_cols = ['fare_amount', 'trip_distance', 'total_amount']df = df[(df[non_zero_cols] > 0).all(axis=1)]

### 3.5 Outlier Handling (your original IQR code — unchanged)

In [8]:
def detect_outliers(df, col):    Q1 = df[col].quantile(0.25)    Q3 = df[col].quantile(0.75)    IQR = Q3 - Q1    lower = Q1 - 1.5 * IQR    upper = Q3 + 1.5 * IQR    outliers = df[(df[col] < lower) | (df[col] > upper)]    return len(outliers), lower, upperoutlier_cols = ['trip_distance', 'fare_amount']for col in outlier_cols:    count, lower, upper = detect_outliers(df, col)    print(f"{col}: {count} outliers (lower: {lower:.2f}, upper: {upper:.2f})")    Q1 = df[col].quantile(0.25)    Q3 = df[col].quantile(0.75)    IQR = Q3 - Q1    df = df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]print(f"Rows after outlier removal: {len(df)}")

### 3.6 Categorical Mapping

Convert numeric codes into meaningful strings.

In [9]:
vendor_dict = {1: 'Creative Mobile', 2: 'Curb Mobility', 6: 'Myle Tech', 7: 'Helix'}ratecode_dict = {1: 'Standard', 2: 'JFK', 3: 'Newark', 4: 'Nassau/Westchester', 5: 'Negotiated', 6: 'Group', 99: 'Unknown'}payment_dict = {0: 'Flex Fare', 1: 'Credit', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Unknown', 6: 'Voided'}df['VendorID'] = df['VendorID'].map(vendor_dict)df['RatecodeID'] = df['RatecodeID'].map(ratecode_dict)df['payment_type'] = df['payment_type'].map(payment_dict)

## 4. Feature Engineering

Create new columns for trip duration, day/hour information, and speed.

In [10]:
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60df['day_of_week'] = df['tpep_pickup_datetime'].dt.day_name()df['hour_of_day'] = df['tpep_pickup_datetime'].dt.hourday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']# New: average speed (mph)df['avg_speed_mph'] = df['trip_distance'] / (df['trip_duration'] / 60)# Flag unrealistic speedsspeed_outliers = df[(df['avg_speed_mph'] > 100) | (df['avg_speed_mph'] < 1)]print(f"Unrealistic speed trips: {len(speed_outliers)}")# Optionally remove themdf = df[(df['avg_speed_mph'] <= 100) & (df['avg_speed_mph'] >= 1)]print(f"Avg speed (mph): {df['avg_speed_mph'].mean().round(2)}")

## 5. Location Zone Analysis (NEW)

Map pickup/dropoff IDs to boroughs and zones, then explore hotspots.

In [11]:
zone_lookup = pd.read_csv('data/taxi+_zone_lookup.csv')df = df.merge(zone_lookup.rename(columns={'LocationID':'PULocationID','Zone':'PU_Zone','Borough':'PU_Borough'}), on='PULocationID', how='left')df = df.merge(zone_lookup.rename(columns={'LocationID':'DOLocationID','Zone':'DO_Zone','Borough':'DO_Borough'}), on='DOLocationID', how='left')# Top 10 pickup zones by frequencytop_pu_zones = df['PU_Zone'].value_counts().head(10)print(top_pu_zones)plt.figure(figsize=(10,5))sns.barplot(x=top_pu_zones.values, y=top_pu_zones.index, palette='crest')plt.title('Top 10 Pickup Zones (Jan 2025)')plt.xlabel('Number of Trips')plt.ylabel('Pickup Zone')plt.show()# Borough pickup distributionborough_counts = df['PU_Borough'].value_counts()print(borough_counts)plt.figure(figsize=(6,6))plt.pie(borough_counts, labels=borough_counts.index, autopct='%1.1f%%', startangle=140)plt.title('Pickup Distribution by Borough (Jan 2025)')plt.axis('equal')plt.show()

## 6. Exploratory Data Analysis (EDA)

Analyze the cleaned and enriched dataset.

### 6.1 Statistical Summary

In [12]:
df.describe()

### 6.2 Time-Based Analysis

In [13]:
pivot_volume = df.pivot_table(index='hour_of_day', columns='day_of_week', values='VendorID', aggfunc='count')[day_order]sns.heatmap(pivot_volume, cmap='viridis', linewidths=0.5)plt.title('Trip Volume by Hour and Day (Jan 2025)')plt.xlabel('Day of Week')plt.ylabel('Hour of Day')plt.show()

### 6.3 Revenue Analysis

In [14]:
pivot_revenue = df.pivot_table(index='hour_of_day', columns='day_of_week', values='total_amount', aggfunc='sum')[day_order]sns.heatmap(pivot_revenue, cmap='magma', linewidths=0.5)plt.title('Revenue by Hour and Day (Jan 2025)')plt.xlabel('Day of Week')plt.ylabel('Hour of Day')plt.show()fare_per_mile = round(df['fare_amount'].sum() / df['trip_distance'].sum(), 2)print(f"Average fare per mile: ${fare_per_mile}")

### 6.4 Passenger Behavior

In [15]:
df = df[df['passenger_count'].between(1, 6)]print(df['passenger_count'].value_counts())print(df.groupby('passenger_count')['trip_distance'].mean().round(2))

### 6.5 Payment Type Analysis

In [16]:
payment_counts = df['payment_type'].value_counts()sns.barplot(x=payment_counts.index, y=payment_counts.values)plt.title('Payment Type Distribution (Jan 2025)')plt.xlabel('Payment Type')plt.ylabel('Number of Trips')plt.xticks(rotation=45)plt.show()

### 6.6 Tip Analysis by Payment Method (NEW)

In [17]:
tip_by_type = df.groupby('payment_type')['tip_amount'].mean().round(2)print(tip_by_type)sns.barplot(x=tip_by_type.index, y=tip_by_type.values, palette='muted')plt.title('Average Tip by Payment Type')plt.xlabel('Payment Method')plt.ylabel('Average Tip ($)')plt.xticks(rotation=45)plt.show()

### 6.7 Trip Characteristics

In [18]:
print(df['trip_distance'].describe())sns.histplot(df['trip_distance'], bins=50, kde=True)plt.title('Trip Distance Distribution (Jan 2025)')plt.xlabel('Distance (miles)')plt.ylabel('Frequency')plt.xlim(0, df['trip_distance'].quantile(0.99))plt.show()top5_trips = df.sort_values('trip_distance', ascending=False).head(5)print(top5_trips[['VendorID','tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count','trip_distance','fare_amount','total_amount','PULocationID','DOLocationID']])

## 7. Conclusion & Next Steps

- We cleaned the data thoroughly—handled missing, negative, zero, and outlier values (IQR kept exactly).  
- Feature engineering added trip duration, average speed, and time breakdowns.  
- We mapped pickup/dropoff zones, identified high-volume areas and borough distribution.  
- EDA showed clear patterns: peak hours, revenue trends, and passenger/payment behaviors.  
- Tip behavior varies by payment method—revealing interesting customer preferences.

**Next steps:**  
- Add a summary of business implications (e.g., resource allocation, pricing strategies).  
- Optionally combine multiple months, or compare with Green Taxi.  
- Explore geo heatmaps or integrate weather/events for deeper insight.

Great work! This notebook is now fully aligned with the project guide and highly portfolio-ready.